# Panalyt Data Engineering Assignment 

By [Shubhanshu Gupta](https://shubhanshugupta.com/contact/)

In [1]:
import pandas as pd
import urllib.request
import json
import numpy as np

### Reading the excel file

In [2]:
data = pd.ExcelFile('data_task.xlsx')

### Reading the two sheets of excel file

In [3]:
job = pd.read_excel(data, 'job')
salary = pd.read_excel(data, 'salary')

In [4]:
job.head()

,Employee,Date,Location,Division,Emp ID
0,XXX501004YYY,2018-08-20,M-KL,M,XXX501004
1,XXX212468YYY,2018-08-06,I-HA,I,XXX212468
2,XXX212468YYY,2018-10-09,I-A,I,XXX212468
3,XXX212270YYY,2017-11-01,I-HA,I,XXX212270
4,XXX212270YYY,2018-10-09,I-A,I,XXX212270


In [5]:
salary.head()

,Employee,Effective Date,Salary,Paid Per,Employee Number
0,XXX501004YYY,2018-08-20,MYR 7800,Month,XXX501004
1,XXX501004YYY,2019-04-01,MYR 8640,Month,XXX501004
2,XXX212468YYY,2018-08-06,IDR 4800000,Month,XXX212468
3,XXX212468YYY,2019-01-01,IDR 5400000,Month,XXX212468
4,XXX212270YYY,2017-11-01,IDR 4800000,Month,XXX212270


### Splitting the location in Job dataframe to country and office. Subsequently, dropped the location column

In [6]:
job[['country','office']] = job['Location'].str.split('-',expand=True)

In [7]:
job.drop('Location', axis=1, inplace=True)
job.head()

,Employee,Date,Division,Emp ID,country,office
0,XXX501004YYY,2018-08-20,M,XXX501004,M,KL
1,XXX212468YYY,2018-08-06,I,XXX212468,I,HA
2,XXX212468YYY,2018-10-09,I,XXX212468,I,A
3,XXX212270YYY,2017-11-01,I,XXX212270,I,HA
4,XXX212270YYY,2018-10-09,I,XXX212270,I,A


### Splitting the Salary in salary dataframe to currency and salary. Subsequently, dropped the original Salary column

In [8]:
salary[['currency','salary']] = salary['Salary'].str.split(' ',expand=True)
salary.head()

,Employee,Effective Date,Salary,Paid Per,Employee Number,currency,salary
0,XXX501004YYY,2018-08-20,MYR 7800,Month,XXX501004,MYR,7800
1,XXX501004YYY,2019-04-01,MYR 8640,Month,XXX501004,MYR,8640
2,XXX212468YYY,2018-08-06,IDR 4800000,Month,XXX212468,IDR,4800000
3,XXX212468YYY,2019-01-01,IDR 5400000,Month,XXX212468,IDR,5400000
4,XXX212270YYY,2017-11-01,IDR 4800000,Month,XXX212270,IDR,4800000


In [9]:
salary.drop('Salary', axis=1, inplace=True)
salary.head()

,Employee,Effective Date,Paid Per,Employee Number,currency,salary
0,XXX501004YYY,2018-08-20,Month,XXX501004,MYR,7800
1,XXX501004YYY,2019-04-01,Month,XXX501004,MYR,8640
2,XXX212468YYY,2018-08-06,Month,XXX212468,IDR,4800000
3,XXX212468YYY,2019-01-01,Month,XXX212468,IDR,5400000
4,XXX212270YYY,2017-11-01,Month,XXX212270,IDR,4800000


## Function to fetch current exchange rates

I have created a dictionary here to store the exchange rate of the currency against USD. The idea is to store the exchange rates once and then use them (from dictionary) to convert the salary in local currency to USD.
Used https://exchangeratesapi.io/ API for fetching exchange rates.

In [10]:
usd_exchange_rates = dict()
def get_usd_exchange_rates(base_currency):
    try:
        if base_currency not in usd_exchange_rates:
            contents = urllib.request.urlopen("https://api.exchangeratesapi.io/latest?base="+str(base_currency)+"&symbols=USD").read()
            exchange_rates = json.loads(contents.decode('UTF-8'))
            usd_exchange_rates[base_currency] = exchange_rates['rates']['USD']            
    except Exception as e:
        print("Error in fetching exchange rate. " + str(e))
        


In [11]:
salary.apply(lambda x: get_usd_exchange_rates(x['currency']), axis=1)
usd_exchange_rates

{'MYR': 0.2431293752,
 'IDR': 7.12454e-05,
 'PHP': 0.0195503744,
 'THB': 0.0323464313,
 'SGD': 0.735105827}

## Function to convert local currency into USD

In [12]:
def get_salary_in_usd(currency, salary):
    return round(usd_exchange_rates[currency]*float(salary), 2)

In [13]:
# Example
get_salary_in_usd('MYR', '2800')

680.76

### Apply the method to the entire column

In [14]:
salary['usd_salary'] = salary.apply(lambda x: get_salary_in_usd(x['currency'] ,x['salary']), axis=1)

### Filling all NAs with Month value

In [15]:
salary['Paid Per'].fillna('Month', inplace=True)

### Converting the daily salary to monthly salary

In [16]:
salary.loc[salary['Paid Per']=='Day', 'usd_salary'] = salary.loc[salary['Paid Per']=='Day']['usd_salary']*22

In [17]:
salary.loc[salary['Paid Per']=='Day', 'Paid Per'] = 'Month'

In [18]:
salary.head()


,Employee,Effective Date,Paid Per,Employee Number,currency,salary,usd_salary
0,XXX501004YYY,2018-08-20,Month,XXX501004,MYR,7800,1896.41
1,XXX501004YYY,2019-04-01,Month,XXX501004,MYR,8640,2100.64
2,XXX212468YYY,2018-08-06,Month,XXX212468,IDR,4800000,341.98
3,XXX212468YYY,2019-01-01,Month,XXX212468,IDR,5400000,384.73
4,XXX212270YYY,2017-11-01,Month,XXX212270,IDR,4800000,341.98


In [19]:
job.head()

,Employee,Date,Division,Emp ID,country,office
0,XXX501004YYY,2018-08-20,M,XXX501004,M,KL
1,XXX212468YYY,2018-08-06,I,XXX212468,I,HA
2,XXX212468YYY,2018-10-09,I,XXX212468,I,A
3,XXX212270YYY,2017-11-01,I,XXX212270,I,HA
4,XXX212270YYY,2018-10-09,I,XXX212270,I,A


In [20]:
job.shape

(3815, 6)

In [21]:
job.loc[job['Employee']=='XXX212468YYY']

,Employee,Date,Division,Emp ID,country,office
1,XXX212468YYY,2018-08-06,I,XXX212468,I,HA
2,XXX212468YYY,2018-10-09,I,XXX212468,I,A


In [22]:
salary.loc[salary['Employee']=='XXX212468YYY']

,Employee,Effective Date,Paid Per,Employee Number,currency,salary,usd_salary
2,XXX212468YYY,2018-08-06,Month,XXX212468,IDR,4800000,341.98
3,XXX212468YYY,2019-01-01,Month,XXX212468,IDR,5400000,384.73


### Doing an outer join on Job and Salary dataframes on Employee and Date columns. Retaining only Employee, Date and USD salary columns from Salary dataframe

In [23]:
merged_df = pd.merge(job, salary[['Employee','Effective Date', 'usd_salary']], how='outer', left_on=['Employee', 'Date'], right_on = ['Employee', 'Effective Date'])


In [24]:
merged_df.head()

,Employee,Date,Division,Emp ID,country,office,Effective Date,usd_salary
0,XXX501004YYY,2018-08-20,M,XXX501004,M,KL,2018-08-20,1896.41
1,XXX212468YYY,2018-08-06,I,XXX212468,I,HA,2018-08-06,341.98
2,XXX212468YYY,2018-10-09,I,XXX212468,I,A,NaT,NaN
3,XXX212270YYY,2017-11-01,I,XXX212270,I,HA,2017-11-01,341.98
4,XXX212270YYY,2018-10-09,I,XXX212270,I,A,NaT,NaN


## Assumptions

1. If an employee has record in Salary dataframe for an effective date and no record in Job dataframe for the same effective date, I have assumed that the employee was in the same division, country and office as in the preceeding row.
2. If an employee has a record in Job dataframe for an effective date and no record in Salary dataframe for the same effective date, I have assumed that the employee was on same salary as before (last drawn salary of the most recent effective date). 

### Sorting the merged dataframe by employee and date column. Doing this because I want to forward fill the NAs with the values in preceeding rows. 

In [25]:
merged_df = merged_df.sort_values(['Employee', 'Date']).reset_index(drop=True)

In [26]:
# Example 1
merged_df.loc[merged_df['Employee']=='XXX101549YYY']

,Employee,Date,Division,Emp ID,country,office,Effective Date,usd_salary
975,XXX101549YYY,2016-09-01,R,XXX101549,T,HQ,2016-09-01,1868.01
976,XXX101549YYY,2018-04-01,R,XXX101549,T,HQ,2018-04-01,2377.46
977,XXX101549YYY,2018-04-20,R,XXX101549,T,HQ,NaT,NaN
978,XXX101549YYY,2018-07-17,R,XXX101549,T,HQ,NaT,NaN
979,XXX101549YYY,NaT,NaN,NaN,NaN,NaN,2017-04-01,1924.05
980,XXX101549YYY,NaT,NaN,NaN,NaN,NaN,2019-04-01,2472.56


In [27]:
# Example 2
merged_df.loc[merged_df['Employee']=='XXX212468YYY']

,Employee,Date,Division,Emp ID,country,office,Effective Date,usd_salary
3939,XXX212468YYY,2018-08-06,I,XXX212468,I,HA,2018-08-06,341.98
3940,XXX212468YYY,2018-10-09,I,XXX212468,I,A,NaT,NaN
3941,XXX212468YYY,NaT,NaN,NaN,NaN,NaN,2019-01-01,384.73


## Following Assumption number 1. 
If there's no value in Date field from Job dataframe (NAs as a result of join), I am filling the effective date from salary dataframe. 

In [28]:
merged_df.loc[merged_df['Date'].isnull(),'Date'] = merged_df['Effective Date']

## Following Assumption number 2. 
If there's no value in Date field from Salary dataframe (NAs as a result of join), I am filling the effective date from job dataframe. 

In [29]:
merged_df.loc[merged_df['Effective Date'].isnull(),'Effective Date'] = merged_df['Date']

## Forward filling the NAs 

In [30]:
merged_df = merged_df.fillna(method='ffill')

In [31]:
merged_df

,Employee,Date,Division,Emp ID,country,office,Effective Date,usd_salary
0,XXX101000YYY,2014-07-01,T,XXX101000,T,HQ,2014-07-01,NaN
1,XXX101000YYY,2018-02-14,T,XXX101000,T,HQ,2018-02-14,NaN
2,XXX101000YYY,2012-07-01,T,XXX101000,T,HQ,2012-07-01,2109.25
3,XXX101000YYY,2017-04-01,T,XXX101000,T,HQ,2017-04-01,2172.55
4,XXX101000YYY,2018-04-01,T,XXX101000,T,HQ,2018-04-01,2227.86
5,XXX101000YYY,2019-04-01,T,XXX101000,T,HQ,2019-04-01,2258.59
6,XXX101001YYY,2012-11-01,T,XXX101001,T,HQ,2012-11-01,1428.90
7,XXX101001YYY,2017-04-01,T,XXX101001,T,HQ,2017-04-01,1470.39
8,XXX101001YYY,2017-07-01,T,XXX101001,T,HQ,2017-07-01,1597.91
9,XXX101001YYY,2018-06-01,T,XXX101001,T,HQ,2018-06-01,1597.91
